In [6]:
# 환경 변수에서 API 키 가져오기

import os
from urllib.parse import unquote

API_KEY = unquote(os.environ['ANIMAL_API_KEY'])

## 라이브러리 사용

In [7]:
# requests 사용

import requests

requests.__version__

'2.21.0'

In [8]:
# sqlite3 사용

import sqlite3

sqlite3.version

'2.6.0'

## 동물보호관리시스템 유기동물 조회 서비스 API 사용
DB에 꽉 채워넣읍시다.

In [9]:
# 동물보호관리시스템 유기동물 조회 서비스 API

# 물음표(?) 앞부분
url = 'http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic'

# 물음표(?) 뒷부분
payload = {
    'serviceKey': API_KEY,
    'bgnde': '20190801',
    'endde': '20191031',
    'numOfRows': 100_000,
}

# API 호출
response = requests.get(url, params=payload)

print(response)

KeyboardInterrupt: 

In [ ]:
# Response의 Text 길이를 확인합니다.
# 만약 이 값이 너무 작다면 오류가 발생했는지 확인해 보세요.

len(response.text)

In [7]:
# XML 데이터 파싱

import xml.etree.ElementTree as ET

root = ET.fromstring(response.text)

root

<Element 'response' at 0x000002A2F6DC5548>

In [8]:
# 컬럼 목록 얻기

item = next(root.iter('item'))

columns = []

for child in item:
    columns.append(child.tag)

columns

['age',
 'careAddr',
 'careNm',
 'careTel',
 'chargeNm',
 'colorCd',
 'desertionNo',
 'filename',
 'happenDt',
 'happenPlace',
 'kindCd',
 'neuterYn',
 'noticeEdt',
 'noticeNo',
 'noticeSdt',
 'officetel',
 'orgNm',
 'popfile',
 'processState',
 'sexCd',
 'specialMark',
 'weight']

## 사용할 컬럼 선택
어떤 컬럼을 사용할지 선택해서 테이블을 만듭시다.

저는 age, colorCd, happenDt, kindCd, orgNm, sexCd, weight만 사용하겠습니다.

In [9]:
columns = ['age', 'colorCd', 'happenDt', 'kindCd', 'orgNm', 'sexCd', 'weight']

## Identifier (고유 식별자)
각 동물 데이터에 대한 Identifier를 만들면 개별적으로 관리하기 좋습니다.

학급에서 학생들에게 번호를 지정하면 같은 이름을 가진 학생을 구분하기도 좋고, 훨씬 쉽고 빠르게 데이터에 접근할 수 있습니다.

“아샬”이란 학생의 국어 점수를 찾으려면 꽤 어려울 수 있지만, (학생 정보가 번호 순서대로 써있다는 전제로) 32번 학생의 국어 점수는 비교적 쉽게 찾을 수 있죠.

SQLite는 PRIMARY_KEY와 AUTOINCREMENT를 통해 Identifier를 제공합니다.

- CREATE TABLE
- SQLite Primary Key
- SQLite AUTOINCREMENT

## Hash
API에서 이미 Identifier를 제공하고 있다면 그걸 그대로 활용할 수 있습니다.

유기동물 조회 서비스 API는 noticeNo 항목으로 “ㅇㅇ-ㅇㅇ-2019-00000” 같은 값을 제공하는데, 이건 숫자가 아니라서 우리의 DB에 Identifier로 바로 활용할 수 없습니다.

그래서 Python은 hash 함수를 제공합니다.

### Hash 쓰는 이유
100 MB 짜리 파일 2개 (A, B)를 비교하려고 하면 모의고사 채점하듯이 하나씩 비교해야한다 -> 오래 걸린다

Hash (25 digit, 256 bit = 32byte) 값을 발급한다 
훨씬 작은 메모리만 비교해서 같은 파일인지 확인할 수 있도록

메모리가 작기 때문에 충돌을 피할 수는 없다 -> 충돌이 적은 방법을 쓴다

### Hash 쓰는 팁
NoticeNm 를 그대로 안 쓴 이유
- 문자열은 자리를 많이 차지하기 때문에 숫자로만 이루어진 hash 값으로 변환한다

id 로 쓸 수 있는 컬럼이 원래 데이터셋에 없는 경우
- 해당 행 전체를 조인한 후 Hash 를 적용한다
- 이런 경우 같은 데이터에 대해 값이 바뀌는 컬럼(Status: 보호중/입양 등)이 있다면 그 컬럼을 제외하고 join -hash 적용해준다

In [1]:
# 문자열을 숫자로 변환

hash('ㅇㅇ-ㅇㅇ-2019-00000')

363122055425167021

In [2]:
hash('ㅇㅇ-ㅇㅇ-2019-00001')

7490902738612944889

In [3]:
hash('ㅇㅇ-ㅇㅇ-2019-00002')

1649265270469203919

In [5]:
# hash 값을 양수로만 쓰고싶은 경우
# abs 함수를 쓰면 절대값을 구할 수 있습니다.

abs(hash('ㅇㅇ-ㅇㅇ-2019-00002'))

1649265270469203919

## DB 접속

In [10]:
# 파일로 기록되는 데이터베이스에 연결/접속

connection = sqlite3.connect('test.db')

In [11]:
# 커서 얻기

cursor = connection.cursor()

## 테이블 생성

In [12]:
# id INTEGER PRIMARY KEY AUTOINCREMENT
# id 자동으로 발급
# 맨 첫줄엔 이거 쓴다

sql = '''
CREATE TABLE animals (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    age TEXT,
    colorCd TEXT,
    happenDt TEXT,
    kindCd TEXT,
    orgNm TEXT,
    sexCd TEXT,
    weight TEXT
)
'''

cursor.execute(sql)

## XML 파싱

In [13]:
# XML 데이터를 rows에 담기

# rows 준비
rows = []

for item in root.iter('item'):
    # row 준비
    row = {}
    
    # 중복을 방지하기 위해 noticeNo 컬럼을 숫자로 바꿔서 Identifier로 활용합니다.
    # 문자열을 숫자로 바꿀 땐 hash 함수를 사용합니다.
    row['id'] = abs(hash(item.find('noticeNo').text))
    
    # 원하는 컬럼만 얻습니다.
    for column in columns:
        row[column] = item.find(column).text

    # rows에 추가
    rows.append(row)

In [14]:
# rows 갯수 확인

len(rows)

30589

In [15]:
# 데이터 확인

rows[0]

{'age': '2019(년생)',
 'colorCd': '흰색',
 'happenDt': '20191014',
 'kindCd': '[개] 믹스견',
 'orgNm': '경상남도 사천시',
 'sexCd': 'F',
 'weight': '1(Kg)'}

## DB에 데이터 넣기
Identifier로 쓰이는 id를 제외한 나머지 값을 INSERT해야 합니다.

INSERT INTO [테이블 이름] ([필드 목록]) VALUES ([값 목록])

필드 목록을 만들기 위해 우리는 join을 사용할 겁니다.

- str.join
- SQLite Python: Inserting Data

In [16]:
# 데이터의 키를 모아서 쉼표로 이어줍니다.

', '.join(rows[0].keys())

'age, colorCd, happenDt, kindCd, orgNm, sexCd, weight'

In [17]:
# 필드 목록에 fields란 이름을 붙입니다.

fields = ', '.join(rows[0].keys())

# f-string을 사용하면 중괄호({}) 안에 변수 등을 넣어서 아주 간단히 SQL을 만들 수 있습니다.

sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?)'

print(sql)

INSERT INTO animals (age, colorCd, happenDt, kindCd, orgNm, sexCd, weight) VALUES (?, ?, ?, ?, ?, ?, ?)


In [18]:
# 테이블에 데이터 넣기

for row in rows:
    fields = ', '.join(row.keys())
    sql = f'INSERT INTO animals ({fields}) VALUES (?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(sql, list(row.values()))

connection.commit()

## DB에 데이터가 잘 들어갔는지 간단히 확인

In [19]:
# 테이블에서 데이터 얻기

cursor.execute('SELECT * FROM animals')

rows = cursor.fetchall()

In [20]:
# 전부 출력하면 무서울테니 갯수만 확인합니다.

print(len(rows))

30589


In [21]:
# 데이터 하나만 확인합니다.

print(rows[0])

(1, '2019(년생)', '흰색', '20191014', '[개] 믹스견', '경상남도 사천시', 'F', '1(Kg)')


## DB 사용 종료
DB를 사용 후엔 close를 하는 게 좋습니다.

In [22]:
connection.close()